In [10]:
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from pandas.tseries.offsets import DateOffset

# csv_import = 'State_Zhvi_1bedroom.csv'
# csv_import = 'State_Zhvi_2bedroom.csv'
# csv_import = 'State_Zhvi_3bedroom.csv'
# csv_import = 'State_Zhvi_4bedroom.csv'
csv_import = 'State_Zhvi_5BedroomOrMore.csv'

data = pd.read_csv(csv_import, header = None)
data_refined = data.iloc[:,5:]
state_list = data_refined[5].tolist()[1:]

data_refined = data.iloc[:,6:].T

df = data_refined.rename(columns = {0: "Date"})

for index, state in enumerate(state_list):
    df.rename(columns = {index + 1 : state}, inplace = True)
    
df["CA"] = pd.to_numeric(df["CA"])
df["Date"] = pd.to_datetime(df["Date"])
formatted_df = df["Date"].dt.strftime("%m/%d/%y")
df.set_index("Date", inplace = True)
model = sm.tsa.statespace.SARIMAX(df["CA"], order = (0, 1, 1), seasonal_order = (1, 1, 0, 12))
model_fit = model.fit()
future_dates = [df.index[-1] + DateOffset(months = x) for x in range(0,60)]
future_datest_df = pd.DataFrame(index = future_dates[1:], columns = df.columns)
future_df = pd.concat([df,future_datest_df])
future_df['CA_forecast'] = model_fit.predict(start = 291, end = 351, dynamic = True)
# future_df[['CA', 'CA_forecast']].plot(figsize=(12, 8))

for state in state_list[1:]:
    df[state] = pd.to_numeric(df[state])
    model = sm.tsa.statespace.SARIMAX(df[state], order = (0, 1, 1), seasonal_order = (1, 1, 0, 12))
    model_fit = model.fit()
    future_df[state + "_forecast"] = model_fit.predict(start = 291, end = 351, dynamic = True)

future_df.to_csv("forecast_" + csv_import)

6     1996-01-31
7     1996-02-29
8     1996-03-31
9     1996-04-30
10    1996-05-31
         ...    
292   2019-11-30
293   2019-12-31
294   2020-01-31
295   2020-02-29
296   2020-03-31
Name: Date, Length: 291, dtype: datetime64[ns]